
## Problem Definition
Develop a machine learning model to predict customer churn for a telecommunications company.

Objective: To classify whether a customer will churn or not, using their personal and subscription information as input.
Type of Problem: Binary classification.
Evaluation Metric: AUC (Area Under the Curve).
Goal: Improve the predictive performance of the model by enhancing the baseline code.
When running the baseline code, the Public Score (AUC) is: 0.82182.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/빅데이터 강의 /머신러닝(사이버대학교)/과제/"
SEED = 42
import pandas as pd
import numpy as np

train = pd.read_csv(f"{DATA_PATH}customer_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}customer_test.csv") # 테스트 데이터
test_target = pd.read_csv(f"{DATA_PATH}sample_submission.csv") # 테스트데이터 정답값
train.shape , test.shape , test_target.shape

ValueError: mount failed

In [ ]:
train.isnull().sum()

ID                   0
성별                   0
고령자여부                0
기혼여부                 0
부양가족여부               0
가입기간                 0
집전화이용여부              0
다중회선여부               0
인터넷이용방식           1071
인터넷보안서비스사용여부         0
인터넷백업서비스사용여부         0
기기방화벽서비스사용여부         0
인터넷기술지원서비스사용여부       0
스트리밍TV여부             0
스트리밍영화여부             0
약정기간                 0
지로여부                 0
지불방법                 0
한달요금                 0
총지불요금                0
이탈여부                 0
dtype: int64

In [ ]:
test.isnull().sum()

ID                  0
성별                  0
고령자여부               0
기혼여부                0
부양가족여부              0
가입기간                0
집전화이용여부             0
다중회선여부              0
인터넷이용방식           455
인터넷보안서비스사용여부        0
인터넷백업서비스사용여부        0
기기방화벽서비스사용여부        0
인터넷기술지원서비스사용여부      0
스트리밍TV여부            0
스트리밍영화여부            0
약정기간                0
지로여부                0
지불방법                0
한달요금                0
총지불요금               0
dtype: int64

In [ ]:
train["인터넷이용방식"] = train["인터넷이용방식"].fillna("UNK")
test["인터넷이용방식"] = test["인터넷이용방식"].fillna("UNK")

In [ ]:
train.isnull().sum().sum() , test.isnull().sum().sum()

(0, 0)

In [ ]:
train["이탈여부"].mean()

0.2653144016227181

In [ ]:
train_ft = train.iloc[:,1:-1].copy() # ID 및 정답 제외
test_ft = test.iloc[:,1:].copy() # ID 제외
train_ft.shape, test_ft.shape
train_ft.head()

,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷이용방식,인터넷보안서비스사용여부,인터넷백업서비스사용여부,기기방화벽서비스사용여부,인터넷기술지원서비스사용여부,스트리밍TV여부,스트리밍영화여부,약정기간,지로여부,지불방법,한달요금,총지불요금
0,1,0,0,0,23,1,0,광,0,1,0,1,0,0,0,0,메일,79.10,1783.75
1,1,0,0,0,1,1,0,디지털,0,0,0,0,0,0,0,0,메일,45.40,45.40
2,0,0,0,0,23,1,1,광,0,0,1,1,1,1,0,0,메일,104.05,2470.10
3,1,0,0,0,11,1,0,디지털,0,0,1,1,0,1,1,1,신용카드,64.90,697.25
4,1,0,0,0,5,1,1,광,0,0,0,0,1,0,0,0,전자,85.20,474.80


In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.7 MB/s eta 0:00:00


In [ ]:
import category_encoders as ce

In [ ]:
train_ft.columns

Index(['성별', '고령자여부', '기혼여부', '부양가족여부', '가입기간', '집전화이용여부', '다중회선여부', '인터넷이용방식',
       '인터넷보안서비스사용여부', '인터넷백업서비스사용여부', '기기방화벽서비스사용여부', '인터넷기술지원서비스사용여부',
       '스트리밍TV여부', '스트리밍영화여부', '약정기간', '지로여부', '지불방법', '한달요금', '총지불요금'],
      dtype='object')

In [ ]:
cols = ["인터넷이용방식","지불방법"]

enc = ce.one_hot.OneHotEncoder(return_df=True)

# 학습데이터
tmp = enc.fit_transform(train_ft[cols]).add_prefix("onehot_")
train_ft = pd.concat([train_ft,tmp],axis=1)

# 테스트데이터
tmp = enc.transform(test_ft[cols]).add_prefix("onehot_")
test_ft = pd.concat([test_ft,tmp],axis=1)
train_ft.head()

,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷이용방식,인터넷보안서비스사용여부,인터넷백업서비스사용여부,...,지불방법,한달요금,총지불요금,onehot_인터넷이용방식_1,onehot_인터넷이용방식_2,onehot_인터넷이용방식_3,onehot_지불방법_1,onehot_지불방법_2,onehot_지불방법_3,onehot_지불방법_4
0,1,0,0,0,23,1,0,광,0,1,...,메일,79.10,1783.75,1,0,0,1,0,0,0
1,1,0,0,0,1,1,0,디지털,0,0,...,메일,45.40,45.40,0,1,0,1,0,0,0
2,0,0,0,0,23,1,1,광,0,0,...,메일,104.05,2470.10,1,0,0,1,0,0,0
3,1,0,0,0,11,1,0,디지털,0,0,...,신용카드,64.90,697.25,0,1,0,0,1,0,0
4,1,0,0,0,5,1,1,광,0,0,...,전자,85.20,474.80,1,0,0,0,0,1,0


In [ ]:
cols = ["인터넷이용방식","지불방법"]
train_ft = train_ft.drop(columns = cols)
test_ft = test_ft.drop(columns = cols)

train_ft.shape , test_ft.shape

((4930, 24), (2113, 24))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 학습데이터
train_ft = pd.DataFrame(
    scaler.fit_transform(train_ft),
    columns = train_ft.columns
)

# 테스트 데이터
test_ft = pd.DataFrame(
    scaler.transform(test_ft),
    columns = test_ft.columns
)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

In [ ]:
target = train["이탈여부"]
target

0       0
1       1
2       1
3       0
4       1
       ..
4925    0
4926    0
4927    0
4928    0
4929    0
Name: 이탈여부, Length: 4930, dtype: int64

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4930 entries, 0 to 4929
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4930 non-null   object 
 1   성별              4930 non-null   int64  
 2   고령자여부           4930 non-null   int64  
 3   기혼여부            4930 non-null   int64  
 4   부양가족여부          4930 non-null   int64  
 5   가입기간            4930 non-null   int64  
 6   집전화이용여부         4930 non-null   int64  
 7   다중회선여부          4930 non-null   int64  
 8   인터넷이용방식         4930 non-null   object 
 9   인터넷보안서비스사용여부    4930 non-null   int64  
 10  인터넷백업서비스사용여부    4930 non-null   int64  
 11  기기방화벽서비스사용여부    4930 non-null   int64  
 12  인터넷기술지원서비스사용여부  4930 non-null   int64  
 13  스트리밍TV여부        4930 non-null   int64  
 14  스트리밍영화여부        4930 non-null   int64  
 15  약정기간            4930 non-null   int64  
 16  지로여부            4930 non-null   int64  
 17  지불방법            4930 non-null   o

In [ ]:
!pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.5/260.5 kB 10.9 MB/s eta 0:00:00


In [ ]:
from flaml import AutoML

In [ ]:
auto_ml = AutoML()
params = {
    "metric": "roc_auc",
    "task": "classification",
    "time_budget": 60, # 초단위
    "seed": SEED,
    "early_stop":True
}
auto_ml.fit(train_ft, target, **params)

[flaml.automl.logger: 06-08 05:28:27] {1693} INFO - task = classification
[flaml.automl.logger: 06-08 05:28:27] {1700} INFO - Data split method: stratified
[flaml.automl.logger: 06-08 05:28:27] {1703} INFO - Evaluation method: cv
[flaml.automl.logger: 06-08 05:28:27] {1801} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 06-08 05:28:27] {1911} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 06-08 05:28:27] {2221} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 06-08 05:28:27] {2347} INFO - Estimated sufficient time budget=2372s. Estimated necessary time budget=55s.
[flaml.automl.logger: 06-08 05:28:27] {2394} INFO -  at 0.3s,	estimator lgbm's best error=0.1975,	best estimator lgbm's best error=0.1975
[flaml.automl.logger: 06-08 05:28:27] {2221} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 06-08 05:28:28] {2394} INFO -  at 0.4s,	estimator lgbm's best error=0

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarnin

[flaml.automl.logger: 06-08 05:28:29] {2394} INFO -  at 1.8s,	estimator xgboost's best error=0.1980,	best estimator lgbm's best error=0.1526
[flaml.automl.logger: 06-08 05:28:29] {2221} INFO - iteration 10, current learner xgboost


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarnin

[flaml.automl.logger: 06-08 05:28:29] {2394} INFO -  at 1.9s,	estimator xgboost's best error=0.1721,	best estimator lgbm's best error=0.1526
[flaml.automl.logger: 06-08 05:28:29] {2221} INFO - iteration 11, current learner xgboost


...

[flaml.automl.logger: 06-08 05:29:27] {2394} INFO -  at 59.9s,	estimator lgbm's best error=0.1507,	best estimator lgbm's best error=0.1507
[flaml.automl.logger: 06-08 05:29:27] {2221} INFO - iteration 92, current learner lrl1


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 06-08 05:29:27] {2394} INFO -  at 60.2s,	estimator lrl1's best error=0.1694,	best estimator lgbm's best error=0.1507


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.automl.logger: 06-08 05:29:29] {2630} INFO - retrain lgbm for 1.7s
[flaml.automl.logger: 06-08 05:29:29] {2633} INFO - retrained model: LGBMClassifier(colsample_bytree=0.7776783613176049,
               learning_rate=0.056768842449641574, max_bin=31,
               min_child_samples=27, n_estimators=459, num_leaves=4,
               reg_alpha=0.010767168653883007, reg_lambda=156.96800382073496,
               verbose=-1)
[flaml.automl.logger: 06-08 05:29:29] {1941} INFO - fit succeeded
[flaml.automl.logger: 06-08 05:29:29] {1942} INFO - Time taken to find the best model: 42.87995171546936


In [ ]:
auto_ml.model.estimator

LGBMClassifier(colsample_bytree=0.7776783613176049,
               learning_rate=0.056768842449641574, max_bin=31,
               min_child_samples=27, n_estimators=459, num_leaves=4,
               reg_alpha=0.010767168653883007, reg_lambda=156.96800382073496,
               verbose=-1)

In [ ]:
auto_ml.best_config

{'n_estimators': 459,
 'num_leaves': 4,
 'min_child_samples': 27,
 'learning_rate': 0.056768842449641574,
 'log_max_bin': 5,
 'colsample_bytree': 0.7776783613176049,
 'reg_alpha': 0.010767168653883007,
 'reg_lambda': 156.96800382073496}

In [ ]:
auto_ml.best_loss

0.15066934610535326

In [ ]:
1 - auto_ml.best_loss

0.8493306538946468

In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.model_selection import cross_val_score
model = LGBMClassifier(random_state=SEED, **auto_ml.best_config) # 튜닝된 하이퍼파라미터  적용
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='roc_auc',n_jobs = -1)
print(scores) # 폴드별 검증점수 리스트
np.mean(scores) # cv 평균 점수

[0.84865165 0.84948705 0.85103973 0.8435179  0.85204378]


0.8489480227113709

In [ ]:
model.fit(train_ft,target)
pred = auto_ml.predict_proba(test_ft)[:,1]
pred[:5]

array([0.69361482, 0.11278581, 0.49125283, 0.0775742 , 0.62933175])

In [ ]:
submit = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
2108,test_2108,0.5
2109,test_2109,0.5
2110,test_2110,0.5
2111,test_2111,0.5


In [ ]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.693615
1,test_1,0.112786
2,test_2,0.491253
3,test_3,0.077574
4,test_4,0.629332
...,...,...
2108,test_2108,0.694903
2109,test_2109,0.054869
2110,test_2110,0.013360
2111,test_2111,0.019764


In [ ]:
submit.to_csv("test_submit.csv",index=False) # 인덱스는 제외하기 위해 False